In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import os
import sys
sys.path.append(os.path.abspath(".."))


In [2]:
#with open("../intermediate_files/test_charge.pkl", "rb") as f:
with open("../intermediate_files/test_gamma1_primary.pkl", "rb") as f:
    outputs = pickle.load(f)

true_gamma_info_df = outputs[0]
downsampled_Tcluster_spacepoints_with_charge = outputs[1]
real_nu_reco_nu_downsampled_spacepoints = outputs[2]
real_nu_reco_cosmic_downsampled_spacepoints = outputs[3]
real_cosmic_reco_nu_downsampled_spacepoints = outputs[4]
real_cosmic_reco_cosmic_downsampled_spacepoints = outputs[5]
real_gamma1_downsampled_spacepoints = outputs[6]
real_gamma2_downsampled_spacepoints = outputs[7]
real_other_particles_downsampled_spacepoints = outputs[8]
real_cosmic_downsampled_spacepoints = outputs[9]


In [3]:
reco_nu_vtx = []
true_nu_vtx = []
true_gamma1_pairconvert_vtx = []
true_gamma2_pairconvert_vtx = []

for event_i in range(len(true_gamma_info_df)):

    reco_nu_vtx.append([true_gamma_info_df["reco_nu_vtx_x"][event_i], true_gamma_info_df["reco_nu_vtx_y"][event_i], true_gamma_info_df["reco_nu_vtx_z"][event_i]])
    true_nu_vtx.append([true_gamma_info_df["true_nu_vtx_x"][event_i], true_gamma_info_df["true_nu_vtx_y"][event_i], true_gamma_info_df["true_nu_vtx_z"][event_i]])
   
    if len(true_gamma_info_df["true_gamma_pairconversion_xs"][event_i]) > 0:
        true_gamma1_pairconvert_vtx.append([true_gamma_info_df["true_gamma_pairconversion_xs"][event_i][0], true_gamma_info_df["true_gamma_pairconversion_ys"][event_i][0], true_gamma_info_df["true_gamma_pairconversion_zs"][event_i][0]])
    else:
        true_gamma1_pairconvert_vtx.append(None)
    if len(true_gamma_info_df["true_gamma_pairconversion_xs"][event_i]) > 1:
        true_gamma2_pairconvert_vtx.append([true_gamma_info_df["true_gamma_pairconversion_xs"][event_i][1], true_gamma_info_df["true_gamma_pairconversion_ys"][event_i][1], true_gamma_info_df["true_gamma_pairconversion_zs"][event_i][1]])
    else:
        true_gamma2_pairconvert_vtx.append(None)

# swap the gamma1 and gamma2 pairconvert vtxs if the second one is higher energy
for event_i in range(len(true_gamma1_pairconvert_vtx)):
    if true_gamma2_pairconvert_vtx[event_i] is not None and true_gamma1_pairconvert_vtx[event_i] is not None:
        if true_gamma_info_df["true_gamma_energies"][event_i][1] > true_gamma_info_df["true_gamma_energies"][event_i][0]:
            true_gamma1_pairconvert_vtx[event_i], true_gamma2_pairconvert_vtx[event_i] = true_gamma2_pairconvert_vtx[event_i], true_gamma1_pairconvert_vtx[event_i]

# if there is only one pairconvert vtx, make it gamma1
for event_i in range(len(true_gamma1_pairconvert_vtx)):
    if true_gamma1_pairconvert_vtx[event_i] is None and true_gamma2_pairconvert_vtx[event_i] is not None:
        true_gamma1_pairconvert_vtx[event_i] = true_gamma2_pairconvert_vtx[event_i]
        true_gamma2_pairconvert_vtx[event_i] = None

if len(true_gamma1_pairconvert_vtx) == 0:
    true_gamma1_pairconvert_vtx = None
if len(true_gamma2_pairconvert_vtx) == 0:
    true_gamma2_pairconvert_vtx = None


In [4]:
for i in range(50):
    if reco_nu_vtx[i][0] != -1:
        print(i, end=", ")

0, 2, 3, 4, 5, 6, 7, 10, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 

In [10]:
from helpers.plotting_3d import plot_event

# event that we failed before
event_index = 61

#event_index = 2

#event_index = 7 # this is weird now

#event_index = 10084

event_index = 17

points_dic = {}

if downsampled_Tcluster_spacepoints_with_charge[event_index] is not None and len(downsampled_Tcluster_spacepoints_with_charge[event_index]) > 0:
    points_dic["Tcluster_spacepoints_with_charge"] = (downsampled_Tcluster_spacepoints_with_charge[event_index][:, :3], downsampled_Tcluster_spacepoints_with_charge[event_index][:, 3], "jet", 2, True)

if real_gamma1_downsampled_spacepoints[event_index] is not None and len(real_gamma1_downsampled_spacepoints[event_index]) > 0:
    points_dic["real_gamma1_downsampled_spacepoints"] = (real_gamma1_downsampled_spacepoints[event_index], "green", None, 2, True)

if real_gamma2_downsampled_spacepoints[event_index] is not None and len(real_gamma2_downsampled_spacepoints[event_index]) > 0:
    points_dic["real_gamma2_downsampled_spacepoints"] = (real_gamma2_downsampled_spacepoints[event_index], "lightgreen", None, 2, True)

if real_other_particles_downsampled_spacepoints[event_index] is not None and len(real_other_particles_downsampled_spacepoints[event_index]) > 0:
    points_dic["real_other_particles_downsampled_spacepoints"] = (real_other_particles_downsampled_spacepoints[event_index], "brown", None, 2, True)

if real_cosmic_downsampled_spacepoints[event_index] is not None and len(real_cosmic_downsampled_spacepoints[event_index]) > 0:
    points_dic["real_cosmic_downsampled_spacepoints"] = (real_cosmic_downsampled_spacepoints[event_index], "blue", None, 2, True)

if reco_nu_vtx[event_index] is not None and len(reco_nu_vtx[event_index]) > 0:
    points_dic["reco_nu_vtx"] = (reco_nu_vtx[event_index], "orange", None, 10, "legendonly")

if true_nu_vtx[event_index] is not None and len(true_nu_vtx[event_index]) > 0:
    points_dic["true_nu_vtx"] = (true_nu_vtx[event_index], "red", None, 10, "legendonly")

if true_gamma1_pairconvert_vtx[event_index] is not None and len(true_gamma1_pairconvert_vtx[event_index]) > 0:
    points_dic["true_gamma1_pairconvert_vtx"] = (true_gamma1_pairconvert_vtx[event_index], "green", None, 10, True)

if true_gamma2_pairconvert_vtx[event_index] is not None and len(true_gamma2_pairconvert_vtx[event_index]) > 0:
    points_dic["true_gamma2_pairconvert_vtx"] = (true_gamma2_pairconvert_vtx[event_index], "lightgreen", None, 10, True)

plot_event(event_index, points_dic)
